In [1]:
import sqlite3
import pandas as pd
import os
import musicbrainzngs

os.chdir('../')

In [2]:
os.getcwd()

'c:\\Users\\cunn2\\OneDrive\\DSML\\Project\\thesis-repo'

### Adding copyright claims table:

In [3]:
TABLE_NAME = 'copyright_cases'
DATABASE = 'data/music.db'
EXCEL_FILE = 'data/thesis-web-data.xlsx'

In [4]:
# # Read the Excel file
# df = pd.read_excel(EXCEL_FILE, sheet_name=0)  # Read the first sheet

# # Connect to SQLite database
# conn = sqlite3.connect(DATABASE)
# cursor = conn.cursor()

# # Get column names from the DataFrame
# columns = df.columns.tolist()

# # Create the table with a primary key
# create_table_query = f'''
# CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
#     id INTEGER PRIMARY KEY AUTOINCREMENT,
#     {', '.join([f'"{col}" TEXT' for col in columns])}
# )
# '''
# cursor.execute(create_table_query)

# # Prepare the INSERT query
# placeholders = ', '.join(['?' for _ in columns])
# insert_query = f'''
# INSERT INTO {TABLE_NAME} ({', '.join([f'"{col}"' for col in columns])})
# VALUES ({placeholders})
# '''

# # Insert the data
# cursor.executemany(insert_query, df.values.tolist())

# # Commit changes and close connection
# conn.commit()
# conn.close()

# print(f"Data imported successfully into table '{TABLE_NAME}' with added primary key")

### Creating song dataset

deletes a table

In [14]:
# # Connect to SQLite database
# conn = sqlite3.connect(DATABASE)
# cursor = conn.cursor()

# cursor.execute('DROP TABLE copyright_songs')

# conn.commit()
# conn.close()

creates song table

In [15]:
# # Connect to SQLite database
# conn = sqlite3.connect(DATABASE)
# cursor = conn.cursor()

# # Create the table with a primary key
# create_table_query = f'''
# CREATE TABLE IF NOT EXISTS copyright_songs (
#     id INTEGER PRIMARY KEY AUTOINCREMENT,
#     musicbrainz_id TEXT UNIQUE,
#     case_artist TEXT,
#     case_title TEXT,
#     case_id INTEGER,
#     mb_artist TEXT,
#     mb_title TEXT,
#     midi_file_path TEXT,
#     encoding_vectors BLOB,
#     FOREIGN KEY (case_id) REFERENCES copyright_cases(id)
# );
# '''
# cursor.execute(create_table_query)

# # Commit changes and close connection
# conn.commit()
# conn.close()


defines music brainz search function

In [16]:
# Set up MusicBrainz API
musicbrainzngs.set_useragent("plagiarism-project", "0.1", "ucabllc@ucl.ac.uk")

# Connect to the database
conn = sqlite3.connect(DATABASE)
cursor = conn.cursor()

# Function to search for a song in MusicBrainz
def search_musicbrainz(artist, title):
    result = musicbrainzngs.search_recordings(artist=artist, recording=title, limit=1)
    if result['recording-list']:
        return result['recording-list'][0]
    return None

In [8]:
r = search_musicbrainz(artist='Kanye', title = 'Stronger')

adds songs from the cases to the table

In [17]:
fetch = f'''
SELECT 
    id, 
    complaining_work as song1, 
    defending_work as song2, 
    complaining_author as artist1, 
    defending as artist2 
FROM copyright_cases
'''
# Get all copyright cases
cursor.execute(fetch)
cases = cursor.fetchall()

for case in cases:
    case_id, title1, title2, artist1, artist2 = case
    # Process Song 1
    song1 = search_musicbrainz(artist1, title1)
    print(f'trying to find {title1}')
    if song1:
        print(f'{song1["title"]} found.')
        cursor.execute("""
            INSERT OR IGNORE INTO copyright_songs (musicbrainz_id, case_artist, case_title, mb_artist, mb_title, case_id)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (song1['id'], artist1, title1, song1['artist-credit-phrase'], song1['title'], case_id))

    # Process Song 2
    song2 = search_musicbrainz(artist2, title2)
    if song2:
        cursor.execute("""
            INSERT OR IGNORE INTO copyright_songs (musicbrainz_id, case_artist, case_title, mb_artist, mb_title, case_id)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (song2['id'], artist2, title2, song2['artist-credit-phrase'], song2['title'], case_id))

conn.commit()
conn.close()

trying to find The Cot Beneath the Hill
Beneath the Hill found.
trying to find The Old Arm Chair
The Old Arm Chair found.
trying to find "Alethia Waltz", based on a theme from Michael Balfe's "The Bohemia Girl"
Main Theme (Lorne Balfe) found.
trying to find Serious Family Polka
Ukrainia Polka found.
trying to find My Own Sweet Darling
My sweet darling found.
trying to find Cradle’s Empty Baby’s Gone
Gone Baby Gone found.
trying to find Dora Dean
Dora Dean found.
trying to find "My Lady's Bower"
Lady's Bower found.
trying to find Kentucky Babe
Called to Serve found.
trying to find Never Turn Back
Turn Back, Turn Back found.
trying to find I Hear You Calling Me
I Hear You Calling Me found.
trying to find You Will Never Know ...
... You Will Never Know found.
trying to find Livery Stable Blues
Livery Stable Blues found.
trying to find Themes from opera "Polly"
The Beggar's Opera: Act I. Air. "Pretty Polly, say" (Macheath, Polly) found.
trying to find Maria Cahill's Arab Love Song
Arab Lov